In [1]:
import numpy as np
import pandas as pd
from linear_regression import LinearRegression

In [14]:
np.random.seed(0)

dataset = pd.read_csv("Houses.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-20 00:00:00,289000.0,3.0,2.50,2090,4700,2.0,0,0,3,2090,0,2002,0,17814 SE 257th St,Covington,WA 98042,USA
1,2014-06-19 00:00:00,429900.0,4.0,2.50,2640,8625,2.0,0,0,3,2640,0,1987,2000,14007 SE 159th Pl,Renton,WA 98058,USA
2,2014-06-04 00:00:00,129000.0,1.0,1.00,650,15364,1.0,0,0,4,650,0,1967,0,13712 SE 256th Pl,Kent,WA 98042,USA
3,2014-05-28 00:00:00,600000.0,4.0,2.00,2510,38141,1.0,0,0,3,2510,0,1960,2012,13507 62nd Ave NE,Kirkland,WA 98034,USA
4,2014-06-03 00:00:00,1298000.0,4.0,3.50,2790,10125,1.5,0,0,5,2790,0,1985,0,4510 89th Ave SE,Mercer Island,WA 98040,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,2014-05-20 00:00:00,440000.0,3.0,1.50,1270,1443,3.0,0,0,3,1270,0,2007,0,9608 Roosevelt Way NE,Seattle,WA 98115,USA
4596,2014-06-24 00:00:00,210000.0,2.0,1.00,970,5500,1.0,0,0,3,970,0,1956,2001,5031 Martin Luther King Jr Way S,Seattle,WA 98118,USA
4597,2014-05-31 00:00:00,538888.0,5.0,2.75,2080,13189,2.0,0,0,3,2080,0,1987,2000,23022 NE 21st St,Sammamish,WA 98074,USA
4598,2014-06-16 00:00:00,1920000.0,4.0,2.50,3070,34412,1.0,0,3,4,2070,1000,1950,1983,3257 Evergreen Point Rd,Medina,WA 98039,USA


In [15]:
dataset["country"].value_counts()

USA    4600
Name: country, dtype: int64

In [16]:
dataset["city"].value_counts()

Seattle                1573
Renton                  293
Bellevue                286
Redmond                 235
Kirkland                187
Issaquah                187
Kent                    185
Auburn                  176
Sammamish               175
Federal Way             148
Shoreline               123
Woodinville             115
Maple Valley             96
Mercer Island            86
Burien                   74
Snoqualmie               71
Kenmore                  66
Des Moines               58
North Bend               50
Covington                43
Duvall                   42
Lake Forest Park         36
Bothell                  33
Newcastle                33
SeaTac                   29
Tukwila                  29
Vashon                   29
Enumclaw                 28
Carnation                22
Normandy Park            18
Clyde Hill               11
Fall City                11
Medina                   11
Black Diamond             9
Ravensdale                7
Pacific             

In [100]:
new_dataset = dataset.copy()
new_dataset = new_dataset.drop(new_dataset.columns[[14, 17]], axis=1)
new_dataset['statezip'] = new_dataset['statezip'].apply(lambda name: name[3:])
new_dataset['date'] = new_dataset['date'].apply(lambda name: name[6])
new_dataset.loc[(new_dataset['yr_renovated'] == 0), 'yr_renovated'] = new_dataset['yr_built']

In [101]:
cities = set(new_dataset["city"])
city_id = 1
for i in cities:
    new_dataset.loc[new_dataset['city'] == i, 'city'] = city_id
    city_id += 1

In [102]:
new_dataset

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
0,5,289000.0,3.0,2.50,2090,4700,2.0,0,0,3,2090,0,2002,2002,18,98042
1,6,429900.0,4.0,2.50,2640,8625,2.0,0,0,3,2640,0,1987,2000,6,98058
2,6,129000.0,1.0,1.00,650,15364,1.0,0,0,4,650,0,1967,1967,27,98042
3,5,600000.0,4.0,2.00,2510,38141,1.0,0,0,3,2510,0,1960,2012,43,98034
4,6,1298000.0,4.0,3.50,2790,10125,1.5,0,0,5,2790,0,1985,1985,32,98040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,5,440000.0,3.0,1.50,1270,1443,3.0,0,0,3,1270,0,2007,2007,29,98115
4596,6,210000.0,2.0,1.00,970,5500,1.0,0,0,3,970,0,1956,2001,29,98118
4597,5,538888.0,5.0,2.75,2080,13189,2.0,0,0,3,2080,0,1987,2000,5,98074
4598,6,1920000.0,4.0,2.50,3070,34412,1.0,0,3,4,2070,1000,1950,1983,10,98039


In [103]:
new_dataset = new_dataset.astype('float')
train = new_dataset[:3680]
test = new_dataset[3680:]

In [104]:
X_train = np.array(train.loc[:, train.columns != 'price'])
y_train_true = np.array(train.loc[:, train.columns == 'price'])

X_test = np.array(test.loc[:, test.columns != 'price'])
y_test_true = np.array(test.loc[:, test.columns == 'price'])

In [105]:
model = LinearRegression()
model.fit(X_train, y_train_true)

In [106]:
y_test_pred = model.predict(X_test)

In [107]:
r_score = model.score(y_test_true, y_test_pred)
print(r_score)

0.5315212256744414


In [108]:
model.coefficients()

array([[ 2.12502038e+04],
       [-5.55672831e+04],
       [ 6.06221557e+04],
       [ 1.54128031e+02],
       [-6.39874400e-01],
       [ 4.44216186e+04],
       [ 2.66778870e+05],
       [ 4.70001529e+04],
       [ 3.77565381e+04],
       [ 8.60108202e+01],
       [ 6.81169671e+01],
       [-2.55136973e+03],
       [ 5.91151838e+02],
       [-8.41508985e+02],
       [-8.34074377e+01],
       [ 1.18328796e+07]])